# AIR(algebraic intermediate representation)

## 1.背景

传统痛点：传统的区块链proof使用的是全部交易重放，每一个节点都需要重新执行验证每笔交易的计算，随着交易量和时间的增长，验证时间大幅提升。

解决办法：证明方从传统的上传交易到**上传证明**，证明方只需要重新计算，检查结果与声称的相符合即可，随着区块链的吞吐量增大，通过可扩展性可以使得验证时间大幅下降。

![截图](images/81e5bf57d6347cd978fb1348a6f61d41.png)

AIR在此为STARK提供了执行轨迹(Execution Trace)的多项式生成。

## 2.概念

STARK 的第一步称为算术化，它是验证计算的问题转换成检查某个多项式的问题。但算术化本身只能将计算完整性的语句转换成多项式

<br/>

算术化本身由两步组成：

第一步是生成执行轨迹和多项式约束。

第二步是将这两个对象转换为单个低次多项式。

<br/>

证明者和验证者在多项式约束上达成一致后，证明者生成一个执行轨迹，证明者会让验证者相信这个执行轨迹上满足多项式约束。

## 3.如何算术化

### 具体示例：

生成执行轨迹与多项式约束

#### 1. 这是一张超市收据

![截图](images/37018e010da414a860c163e7861b5828.png)

<br/>

**执行轨迹(Execution Trace)：**

我们要生成的执行跟踪类型必须具有简洁可测试的特殊特征——每一行只能根据跟踪中靠近它的行进行验证，并且对每一对应用相同的验证过程的行。

添加一行运行中的总和

![截图](images/b3526f670cd61afbb2496f426fe0bc48.png)

此时我们在给出前一行的前提下，可以单独验证每一行。

假设给出第3行，那么我们可以验证第4行： 12.96+2.65=16.41，这就称之为简洁约束。

<br/>

**多项式约束**

|Avocado|4.98|0.00|
|--|--|--|
|Apple|7.98|4.98|
|Milk|3.45|12.96|
|Bread|2.65|16.41|
|BrwonSugar|1.40|19.06|
|Total|20.46|20.46|

用 $A_{i,j}$ 表示第 i 行第 j 列中单元格的值，可以得到:

1. $A_{0,2}=0$
2. $\forall 1 \leq i \leq 5$ : $A_{i,2} - A_{i-1,2} - A_{i-1,1} = 0$
3. $A_{5,1} - A_{5,2} = 0$

<br/>

我们将收据的问题转换为简洁可测试的**执行轨迹(Execution Trace)**，以及一组相对应的多项式约束（如上），且只有原始收据总和正确时以上才会成立。

这些$A_{i,j}$中的线形约束，如果我们使用的的多项式约束度数高的话，会对证明的长度和生成证明的时间产生不利的影响，所以对我们来说线性的约束是最好的。

<br/>

<br/>

<br/>

将执行轨迹改写为多项式，将其扩展到一个大域，使用多项式约束将其转换为另一个只在执行轨迹上是低度的多项式。

#### 2. 斐波那契

正确计算$Z_{96769}$到第512位的斐波那契数列。该序列正确定义为

$a_0$ = 1

$a_1$ = 1

$a_{n+2} = (a_{n+1} + a_n) mod 96769$

我们CI声明(the CI Statement) $a_{511}$=62215

<br/>

首先我们可以简单把512个数字列出来，创建一个**执行轨迹(Execution Trace)**：

![截图](images/5223f12e456097c45ea834714de5af59.png)

我们使用的多项式约束：

1. $A_0 - 1 = 0$
2. $A_0 -1 = 0$
3. $\forall0 \leq i <510: A_{i+2} - A_{i+1} - A_i = 0$
4. $A_{511} - 62215 = 0$

<br/>

**转换成多项式**

我们定义一个最大度数为512的多项式 $f(x)$，然后执行轨迹中的元素就是在某个generator g的幂中求值。

即：$$\forall 0\le i < 512: f(g^i) = A_i$$

用$f(x)$替代$A_i$代表多项式约束，我们得到：

1. $f(0) - 1 = 0$
2. $f(0) -1 = 0$
3. $\forall0 \leq i <510: f(g^{i+2}) - f(g^{i+1}) - f(g^{i}) = 0$
4. $f(g^{511}) - 62215 = 0$

注意：1、2、4是引用单个$f(x)$的约束，我们称之为边界约束

我们也可以把上述多项式约束写成这样：

<br/>

$\forall x \in $ {1, $g$, $g^2$, ..., $g^{509}$}: $f(g^2x) - f(gx) - f(x) = 0$

<br/>

我们可以将x作为执行轨迹中的某一行的，那么下一行就是$gx$, 再下一行就是$g^2x$，前一行是$g^{-1}x$，以此类推。

多项式 $f(g^{i+2}) - f(g^{i+1}) - f(g^{i})$，在执行轨迹中除去最后两行，每一行的结果都为0，即1, $g$, $g^2$, ..., $g^{509}$都是这个多项式的根，且这个多项式的度最多为510，所以我们可以构造一个复合多项式$q(x)$:$$q(x) = \frac {f(g^2x) - f(gx) - f(x)}{\prod^{509}_{i=0}(x-g^i)}$$

但是这个多项式在执行轨迹之内时是一个低度的多项式，但是在之外并不是。

<br/>

**简洁(Succinctness)**

使用少量查询，让验证者对每个查询执行少量计算，且允许在小范围内查询，但是证明者是有可能作恶的。

为了防止这种情况，验证者通过询问三个地方$f(x)$在$w$处的值，得到斐波那契执行轨迹：$f(g^2w)， f(gw)，f(w )$

验证者可以通过计算复合多项式$q(w)$得到在$w$处的值。

分子可以从证明者那里获得的值来计算，但是分母会有可能为0。

由于分母是独立于执行轨迹的，所以可以在计算之前就计算出来，但是实际上执行轨迹可能有数十万，在计算上会付出极大的的代价。

所以我们可以为g的幂设置子群G：$$x^{|G|} -1 = \prod_{g \in G}(x-g)$$

这个等式是正确的，因为两边都是|G|的多项式。其根正是 G 的元素。计算这个等式的右边似乎在|G|中线性时间的操作。但是，如果我们通过平方来求幂(快速幂)，则该等式的左侧可以在对数时间内计算。

之前有问题的斐波那契多项式可以将其重写为：$$\frac {f(g^2w) - f(gw) - f(w)}{\prod^{509}_{i=0}(w-g^i)} = \frac {(w-g^{510})(w-g^{511})*(f(g^2w) - f(gw) - f(w))}{w^{512}-1}$$

重写后的多项式的约束也同样作用于子群。

<br/>

那如何处理多列和多个约束的情况呢，很显然，多列就会有多个多项式，那么这些复合多项式和多项式约束随机线性组合在一起组成一个更大的多项式，成为了STARK最后的一个阶段，低度测试。

<br/>

**总结**

我们已经展示了在给定执行轨迹和约束多项式的情况下，证明者如何构造一个低度多项式，当且仅当原始 CI 语句成立时。此外，我们已经展示了验证者如何有效地查询这个多项式的值，确保证明者没有用一些假的低次多项式替换真正的多项式。

### 资料

[http://medium.com/starkware/stark-math-the-journey-begins-51bd2b063c71](https://medium.com/starkware/stark-math-the-journey-begins-51bd2b063c71)

[https://medium.com/starkware/arithmetization-i-15c046390862](https://medium.com/starkware/stark-math-the-journey-begins-51bd2b063c71)

[https://medium.com/starkware/arithmetization-ii-403c3b3f4355](https://medium.com/starkware/arithmetization-ii-403c3b3f4355)

[https://eprint.iacr.org/2021/582.pdf](https://eprint.iacr.org/2021/582.pdf)
